## Welcome to your notebook.


#### Boilerplate and function defs

In [ ]:
from arcgis.gis import GIS
from mratools import check_incidents, cmp_incidents
gis = GIS("https://msar.maps.arcgis.com", "MRA_Admin") 

In [ ]:
MRA_DEF_PARS={'idx_field':'Incident_UUID', 'date_field':'Incident_Date', 'team_field':'Team'}
OES_DEF_PARS={'idx_field':'Incident_UUID', 'date_field':'Date', 'team_field':'Team'}

def _get_feature_(featureset,idx,val):
    """Get feature(s) that match specific value index"""
    return [feature for feature in featureset.features if feature.as_dict['attributes'][idx] == val]

def _get_features_(featureset, idx, vals):
    """Get feature(s) that match a list of values for an index"""
    return[feature for feature in featureset.features if feature.as_dict['attributes'][idx] in vals ]

def _get_geom_(feature):
    pass


##featureset accessors 
__feature_geom = lambda f: f.geom
__feature_attrs = lambda f: f.as_dict['attributes']
__features = lambda fs: fs.features

def _get_attr_(feature,idx):
    """get value of an attribute for a feature"""
    return __feature_attrs(feature).get(idx) 

def _get_attrs_(featureset, idx):
    """Get values for an attribute in a featureset"""
    return[__feature_attrs(feature).get(idx) for feature in __features(featureset)]

def _get_unique_attrs_(featureset, idx):
    """Get all unique values of an attribute in a featureset"""
    return set(_get_attrs_(featureset,idx=idx))


_nulchk_attr = lambda x: '' in x or None in x
_dupchk_attr = lambda x: len(x) > len(set(x))

def _check(s,tests):
    for test in tests.values():
        yield test(s)
        

def check_incidents(s, idx_field="Incident_UUID",date_field='Incident_Date', team_field="Team", **kwargs):
    """Sanity Check a feature set and return a structured record

    s  - incidents (FeatureSet) to check
    
    0: duplicates T/F- True if there are Duplicates
    1: NULL UUIDs T/F - True if there are Null or empty Incident UUIDS
    """
    
    incident_tests = {"Null Test":_nulchk_attr,
                      "Duplicate Test": _dupchk_attr,
                     }
    attrs = _get_attrs_(s,idx_field)
  
    return (_ for _ in _check(attrs,incident_tests))
    
def check_points(s, idx_field="Linked_Incident_UUID",date_field='Linked_Incident_Date', team_field="Linked_Team", **kwargs):
    """Sanity Check a feature set and return a structured record

    s  - points (FeatureSet) to check
        
    0: duplicates T/F- True if there are Duplicates
    1: NULL UUIDs T/F - True if there are Null or empty Incident UUIDS
    """
    points_tests = {"Null Test":_nulchk_attr,
                      "Duplicate Test": _dupchk_attr,
                     }
    attrs = _get_attrs_(s,idx_field)
  
    return (_ for _ in _check(attrs,points_tests))
    
def cmp_incidents(s1,s2, idx_fields=("Incident_UUID","Incident_UUID",),date_fields=('Incident_Date','Date',), team_fields=("Team","Team",)):
    """Compare FeatureSets of incidents and return a structured record
    
    s1,s2 incidents (FeatureSets) to compare
    idx_field = field for index, can be a list of two elements if s1,s2 index field different.
    date_field = field for index, can be a list of two elements if s1,s2 index field different.
    team_field = field for index, can be a list of two elements if s1,s2 index field different.
    
    returned record structure/tuple:
    idx: test/type - Comment
    
    0: len_check/bool - len(s1)==len(s2)
    1: intersect/int  - number of features in intersection of s1,s2 by idx_field
    2: date_match/int - number of features in intersection of s1,s2 where dates match
    3: team_match/int - number of features in intersection of s1,s2 where the team names match
    4: point_match/int - number of features for all 
    """
    
    len_check = len(s1)==len(s2)
    
    s1_idxs = _get_unique_attrs_(s1, idx_fields[0])
    s2_idxs = _get_unique_attrs_(s2, idx_fields[1])
    
    isect = s1_idxs.intersection(s2_idxs)
    len_isect = len(isect)
    
    s1_ftrs = _get_features_(s1,idx_fields[0], isect)
    s2_ftrs = _get_features_(s2,idx_fields[1], isect)
    
    date_match = 0
    team_match = 0
    point_match = 0
    for feature in s1_ftrs:
        s2_feature, = _get_feature_(s2,idx_fields[1], _get_attr_(feature,idx_fields[0]))
        #print(feature, s2_feature)
        if s2_feature:
            
            if _get_attr_(feature, date_fields[0]) == _get_attr_(s2_feature, date_fields[1]):
                date_match = date_match+1

            if _get_attr_(feature, team_fields[0]) == _get_attr_(s2_feature, team_fields[1]):
                team_match = team_match+1
                
    return len_check, len_isect, date_match, team_match, point_match

                
                
                
                
                
            
    
    
    
    
    
    
    
    

#### Open Feature Service and assign layers

In [ ]:
# Item Added From Toolbar
# Title: MRA Mission Data 3 - 653325 | Type: Feature Service | Owner: MRA_Admin
item = gis.content.get("ceea424c2f2149c38f1cb3be46653325")
oes = gis.content.get("4fe1f4dd817c4f44a993c748e0080437")

#Load Layers
for layer in item.layers:
    print(layer)
    
for layer in oes.layers:
    print(layer)
    
incidents = item.layers[0]
points_found = item.layers[1]

oes_incidents = oes.layers[0]
oes_points = oes.layers[1]

#### June 19 2022 Upload Validation

In [ ]:
#Open UUIDS
#dataset = open('/arcgis/home/CalOES_2H21_IncidentUUIDS.txt')

###Construct select query 
# where Incident_UUID in ('uuid-1', 'uuid2',...))
#select_query_str="Incident_UUID IN ("+ ','.join(['\''+str(_[:-1])+'\'' for _ in dataset])+")"

# where year(Incident_Date = 2021)
select_query_str2="Incident_Date >= '2021-01-01 00:00:00' AND Incident_Date <= '2021-12-31 12:59:59'"
select_query_str3="Date >= '2021-01-01 00:00:00' AND Date <= '2021-12-31 12:59:59' AND MRA_Team IS NOT NULL"


#select on mra dataset
#missions = incidents.query(where = select_query_str)
missions = incidents.query(where = select_query_str2)

#select on oes dataset
#oes_missions = oes.layers[0].query(where=select_query_str)
oes_missions = oes.layers[0].query(where=select_query_str3)

In [ ]:
print("""Quality Verification of OES to MRA Merge for 2021 data executed 6/19/2022

Incident_UUIDS are provided externally in '/arcgis/home/CalOES_2H21_IncidentUUIDS.txt'

selected features in ceea424c2f2149c38f1cb3be46653325 (MRA) : {n_missions}
selected features in 4fe1f4dd817c4f44a993c748e0080437 (OES) : {n_oes}

check mra dataset:::
Duplicate Entries in MRA Feature: {ci_m[0]}
Null/empty Incident ID's in MRA Features: {ci_m[1]}

check oes dataset:::
Duplicate Entries in oes Feature: {ci_o[0]}
Null/empty Incident ID's in oes Features: {ci_o[1]}

compare mra dataset to oes dataset:::
The two datasets Match: {cmp_[0]}
The number of records in both sets (by Incident_UUID) are: {cmp_[1]}
The number of records where the Date matches are: {cmp_[2]}
The number of records where the Team Name matches are: {cmp_[3]}
The number of records where the Geometry matches are: {cmp_[4]}** (not yet implemeted)
""".format( 
        n_missions=len(missions),
        n_oes=len(oes_missions), 
        ci_m=[_ for _ in check_incidents(missions)],
        ci_o=[_ for _ in check_incidents(oes_missions)],
        cmp_=cmp_incidents(missions,oes_missions)
))